# Calc embeddings

Date: 2024/11/25

In [2]:
import sys
sys.path.append("./search")

import sqlite3
import embeddings
import vector_db

In [3]:
CHUNKS_DB_PATH = "../database/chunks.db"
EMBEDDINGS_DB_PATH = "../database/embeddings.db"

In [4]:
# Read all the chunks
with sqlite3.connect(CHUNKS_DB_PATH) as conn:
    cur = conn.cursor()
    records = cur.execute("SELECT id, collection, chunk FROM chunks").fetchall()

In [5]:
data = {}

for r in records:
    collection = r[1]
    if collection not in data:
        data[collection] = []
    data[collection].append([r[0], r[2]])

N = 10  # Batch size for embeddings calculation

In [6]:
records = {} 

# Calculate embeddings
for collection in data.keys():
    print(f"Collection: {collection}")
    items = data[collection]
    for i in range(0, len(items), N):
        ids, chunks = zip(*items[i:i+N])
        vectors = embeddings.get_embedding(chunks)
        if collection not in vectors:
            records[collection] = []
        records[collection].extend(zip(ids, vectors))

# Save the embeddings in a vector database
print("Saving the data in the database...")
for collection, items in records.items():
    col_db = vector_db.VectorDB(EMBEDDINGS_DB_PATH, collection, embeddings.DIMENSION)
    col_db.save(items)

Collection: hansaplatz
Collection: yokohama
Collection: hamburg_station
Collection: takanawa_gateway_station
Collection: dresden_station
Saving the data in the database...


AttributeError: 'sqlite3.Connection' object has no attribute 'enable_load_extension'